In [ ]:
import pysam
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# output consensus_building.ipynb
offset_fasta = pysam.FastaFile("PATH_TO_22_23_PRIMER_BASED_CONSENSUS/H1_consensus.fasta")

# output of PrimalScheme
primer_file = "PATH_TO/primer.bed"

#output primer_checking.sh
variant_file = "OUTPUT_DIRECTORY/H1.23TO24.vcf"

# either ha, na or mp
segment = 'ha'
ref = offset_fasta.references[0]


In [ ]:
def compute_offsets(row):
    start = int(row['start'])
    end = int(row['end'])
    start_offset = offset_fasta.fetch(ref, 0, start).count('X')
    end_offset = offset_fasta.fetch(ref, 0, end).count('X')
    return pd.Series([start_offset, end_offset], index=['start_offset', 'end_offset'])

In [ ]:
vcf = pysam.VariantFile(variant_file)

In [ ]:
primers = pd.read_csv(primer_file, 
                      sep = "\t",
                      skiprows=1,
                      names = ['ref','start','end','name','level_4','strand','sequence'],
                      header=None
                     )

In [ ]:
# Median primer sequence read length
list_of_lengths = (lambda x:[len(i) for i in x])(list(primers['sequence']))
np.median(list_of_lengths)

In [ ]:
primers[[
    "string_name",
    "amplicon_number",
    "position",
    "position_counter"
]] = primers.name.str.split("_", expand=True)

In [ ]:
primers[['start_offset', 'end_offset']] = primers.apply(compute_offsets, axis=1)

In [ ]:
primers_segmet = primers[primers.ref.str.startswith(segment)]
primers_subtype = primers_segmet.copy()  # optionally make a clear independent copy
primers_subtype.loc[:, 'start'] = primers_subtype['start'] - primers_subtype['start_offset']
primers_subtype.loc[:, 'end'] = primers_subtype['end'] - primers_subtype['start_offset']
primers_subtype.loc[:, 'ref'] = ref

In [ ]:
variants = []
with open(variant_file) as f:
    for line in f:
        if line.startswith('#'):
            continue
        parts = line.strip().split('\t')
        pos = int(parts[1])
        info_field = parts[7]
        af_value = 0
        for info in info_field.split(';'):
            if info.startswith('AF='):
                af_str = info.split('=')[1]
                afs = list(map(float, af_str.split(',')))
                af_value = max(afs)
                break
        if af_value > 0.01:
            variants.append((pos, af_value))

def calc_metrics(row):
    start = row['start']
    end = row['end']
    primer_len = end - start + 1
    af_values = [af for (pos, af) in variants if start <= pos <= end]
    mutation_count = len(af_values)
    if mutation_count == 0:
        mut_frequencies = []
    else:
        mut_frequencies = af_values
    return pd.Series([mut_frequencies])

primers_subtype[['mutation_frequencies']] = primers_subtype.apply(calc_metrics, axis=1)

In [ ]:
exploded_df = primers_subtype.explode('mutation_frequencies').reset_index()

# Create the plot
plt.figure(figsize=(15, 6))
sns.boxplot(data=exploded_df, 
               x='name',
               y='mutation_frequencies',
               hue='amplicon_number',
               dodge=True,
               showfliers=False, 
               width=1.5,
               linewidth=1.5,
               boxprops=dict(alpha=0.3))

sns.swarmplot(data=exploded_df,
              x='name',
              y='mutation_frequencies',
              hue='amplicon_number',
              size=3, 
              dodge=True) 

plt.title('Distribution of Mutation Frequencies by Amplicon and Position')
plt.xlabel('Amplicon Number')
plt.ylabel('Mutation Frequency')
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside
plt.tight_layout()

In [ ]:
# sns.set(font_scale=1.1)

exploded_df = primers_subtype.explode('mutation_frequencies').reset_index()

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# LEFT plot
left_data = exploded_df[exploded_df['position'] == 'LEFT']


sns.boxplot(data=left_data,
               x='name',
               y='mutation_frequencies',
               hue='amplicon_number',
               dodge=True,
               showfliers=False, 
               width=1.5,
               linewidth=1.5,
               boxprops=dict(alpha=0.5),
               ax=ax1)

sns.swarmplot(data=left_data,
              x='name',
              y='mutation_frequencies',
              hue='amplicon_number',
              size=5,
              dodge=True,
              ax=ax1) 

ax1.set_title('LEFT Primer Mutation Frequencies',fontsize=16)
ax1.set_xlabel('Primer Name', labelpad=20,fontsize=16)
ax1.set_ylabel('Mutation Frequency',fontsize=16)
ax1.tick_params(axis='x', rotation=90, labelsize=12)
ax1.tick_params(axis='y', labelsize=12)
ax1.set_ylim(0, 1)

sns.move_legend(ax1, "upper center",
                ncol=6,
                title=None, 
                frameon=False,
                fontsize=12
)


# RIGHT plot
right_data = exploded_df[exploded_df['position'] == 'RIGHT']

sns.boxplot(data=right_data,
               x='name',
               y='mutation_frequencies',
               hue='amplicon_number',
               dodge=True,
               showfliers=False, 
               width=1.5,
               linewidth=1.5,
               boxprops=dict(alpha=0.5),
               ax=ax2)

sns.swarmplot(data=right_data,
              x='name',
              y='mutation_frequencies',
              hue='amplicon_number',
              size=5,
              dodge=True,
              ax=ax2) 

ax2.set_title('RIGHT Primer Mutation Frequencies',fontsize=16)
ax2.set_xlabel('Primer Name', labelpad=20,fontsize=16)
ax2.set_ylabel('Mutation Frequency',fontsize=16)
ax2.tick_params(axis='x', rotation=90, labelsize=12)
ax2.tick_params(axis='y', labelsize=12)
ax2.set_ylim(0, 1)

# sns.move_legend(ax2, "upper center",
#                 ncol=6,
#                 title=None, 
#                 frameon=False,
#                 fontsize=12
# )

ax1.get_legend().remove()
ax2.get_legend().remove()
# Create a single legend on the right
handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles, labels,
           loc='lower left',
           bbox_to_anchor=(1.0, 0.4),
           title=None,
           frameon=True,
           fontsize=12)

plt.tight_layout()
plt.subplots_adjust(wspace=0.12)
plt.show()

